In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/cardekho_imputated.csv',index_col =0)

In [3]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [4]:
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
df.drop(['car_name','brand'],axis = 1,inplace=  True)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [8]:
cat_features = [f for f in df.columns if df[f].dtype == 'O']
print(cat_features)
num_features = [f for f in df.columns if df[f].dtype != 'O']
print(num_features)
dis_features = [f for f in num_features if len(df[f].unique()) <= 25]
print(dis_features)

con_features = [f for f in num_features if len(df[f].unique()) > 25]
print(con_features)

['model', 'seller_type', 'fuel_type', 'transmission_type']
['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats', 'selling_price']
['vehicle_age', 'seats']
['km_driven', 'mileage', 'engine', 'max_power', 'selling_price']


In [9]:
from sklearn.model_selection import train_test_split
X = df.drop(['selling_price'],axis = 1)
y = df['selling_price']


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [11]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [12]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
fs = ['seller_type','fuel_type','transmission_type']
num_features.remove('selling_price')
preprocessor = ColumnTransformer(
    [('Standard Scaler',StandardScaler(),num_features),('One Hot Encoding',OneHotEncoder(drop = 'first'),fs)],remainder='passthrough'
)

In [13]:
preprocessor

,transformers,"[('Standard Scaler', ...), ('One Hot Encoding', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [14]:
X = preprocessor.fit_transform(X)

In [15]:
X

array([[ 9.83561835e-01,  1.24733473e+00, -2.75974621e-04, ...,
         1.00000000e+00,  1.00000000e+00,  7.00000000e+00],
       [-3.43933310e-01, -6.90016231e-01, -1.92070558e-01, ...,
         1.00000000e+00,  1.00000000e+00,  5.40000000e+01],
       [ 1.64730941e+00,  8.49241548e-02, -6.47582693e-01, ...,
         1.00000000e+00,  1.00000000e+00,  1.18000000e+02],
       ...,
       [-1.20595237e-02,  2.20538722e-01,  3.44954275e-01, ...,
         0.00000000e+00,  1.00000000e+00,  7.70000000e+01],
       [-3.43933310e-01,  7.25418502e+01, -8.87325923e-01, ...,
         0.00000000e+00,  1.00000000e+00,  1.14000000e+02],
       [-1.33955467e+00, -8.25630799e-01, -4.07839464e-01, ...,
         1.00000000e+00,  0.00000000e+00,  2.50000000e+01]],
      shape=(15411, 14))

In [16]:
X_train,X_test ,y_train ,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [18]:
from sklearn.linear_model import Ridge,Lasso,LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [19]:
def eval_model(y_test,y_pred):
    print("MAE :",mean_absolute_error(y_test,y_pred))
    print("MSE :",mean_squared_error(y_test,y_pred))
    print("r2_Score :",r2_score(y_test,y_pred))

In [20]:
models = {
    "Linear Regression":LinearRegression(),
    "Lasso":Lasso(),
    'Ridge':Ridge(),
    'AdaBoost':AdaBoostRegressor(),
    'XGBoost':XGBRegressor(),
    'GradientBoost':GradientBoostingRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'KNN': KNeighborsRegressor(),
    'Random Forest':RandomForestRegressor()
}

for k in models:
    model = models[k]
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    print(f'**************{k}***************')
    eval_model(y_test,y_pred)
    print()



**************Linear Regression***************
MAE : 284283.44595339836
MSE : 270286925822.7575
r2_Score : 0.6524693637784708

**************Lasso***************
MAE : 284283.89643409953
MSE : 270286253926.42285
r2_Score : 0.652470227692106

**************Ridge***************
MAE : 284241.3092899485
MSE : 270275697193.34164
r2_Score : 0.6524838013718279

**************AdaBoost***************
MAE : 403137.0388185784
MSE : 257647298503.89734
r2_Score : 0.6687211958282644

**************XGBoost***************
MAE : 101211.4375
MSE : 97180909568.0
r2_Score : 0.8750463128089905

**************GradientBoost***************
MAE : 128551.17709354861
MSE : 71422423860.23674
r2_Score : 0.9081661818118844

**************Decision Tree***************
MAE : 134460.35340427374
MSE : 113467867629.71495
r2_Score : 0.8541048180261819

**************KNN***************
MAE : 116040.3127433169
MSE : 104266315244.12794
r2_Score : 0.8659360279341511

**************Random Forest***************
MAE : 104112.844

In [ ]:
rf_params = {
    'n_estimators':[100,50,10],
    'criterion':['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    'max_depth':[None,3,4,6],
    'min_samples_split': [2,4,5],
    'min_samples_leaf': [1,5,3,4]
}
gb_params= {
    'loss':['squared_error', 'absolute_error', 'huber', 'quantile'],
    'learning_rate': [0.1,0,3,0.5,0,8],
    'n_estimators': [100,10,50],
    'criterion': ['friedman_mse', 'squared_error'],
    'max_depth':[3,5,7]
}


In [30]:
models_cv = [
    ('RF',RandomForestRegressor(),rf_params),
    ('GB',GradientBoostingRegressor(),gb_params)
]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
for name,model,params in models_cv:
    model_cv = RandomizedSearchCV(estimator=model,param_distributions=params, n_iter=10,
    cv=3,
    scoring='r2',
    random_state=42
)
    model_cv.fit(X_train,y_train)
    print(model_cv.best_params_)

In [34]:
from sklearn.model_selection import RandomizedSearchCV
params = {
    "n_estimators":[100,50,20,150],
    "max_depth": [4,7,10,12,15],
    "learning_rate": [0.1,0.5,0.4,0.8,0,3],
    "colsample_bytree": [0.9,0.5,0.4]
}
models_cv = [('XgBoost',XGBRegressor(),params)]
for name,model,params in models_cv:
    model_cv = RandomizedSearchCV(estimator=model,param_distributions=params, n_iter=20,cv=3,scoring='r2',random_state=42)

    model_cv.fit(X_train,y_train)
    print(model_cv.best_params_)
    

    y_pred = model_cv.predict(X_test)
    print(f'************{name}************')
    print("MSE : ",mean_absolute_error(y_true=y_test,y_pred=y_pred))
    print("r2_Score : ",r2_score(y_true=y_test,y_pred=y_pred))


c:\Users\hanni\Desktop\NOTES\Semester 5\ML\ml_env\Lib\site-packages\sklearn\metrics\_regression.py:1292: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
c:\Users\hanni\Desktop\NOTES\Semester 5\ML\ml_env\Lib\site-packages\sklearn\metrics\_regression.py:1292: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
c:\Users\hanni\Desktop\NOTES\Semester 5\ML\ml_env\Lib\site-packages\sklearn\metrics\_regression.py:1292: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
c:\Users\hanni\Desktop\NOTES\Semester 5\ML\ml_env\Lib\site-packages\sklearn\model_selection\_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\hanni\Desktop\NOTES\Semester 5\ML\ml_env\Lib\site-packages\sklearn\model_se

{'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.5}
************XgBoost************
MSE :  100602.4765625
r2_Score :  0.9121207594871521


In [36]:
models = {
    'GradientBoost':GradientBoostingRegressor(),
    'Random Forest':RandomForestRegressor(),
    'XGBRegressor':XGBRegressor(n_estimators= 100, max_depth= 7, learning_rate= 0.1, colsample_bytree= 0.5)
}

for k in models:
    model = models[k]
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    print(f'**************{k}***************')
    eval_model(y_test,y_pred)
    print()

**************GradientBoost***************
MAE : 128408.96584131388
MSE : 71002682211.25423
r2_Score : 0.9087058789573365

**************Random Forest***************
MAE : 105512.647383768
MSE : 58602031630.1284
r2_Score : 0.9246504385134491

**************XGBRegressor***************
MAE : 100602.4765625
MSE : 68346798080.0
r2_Score : 0.9121207594871521

